In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

2025-04-22 21:54:51.640354: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
csv_path = "china_mill_data_2025_03_04_09_30_30.csv"
model_path = "china_mill.keras"
window_size = 3360
features = ['energy']

In [ ]:
def load_latest_data(csv_path, features, window_size):
    df = pd.read_csv(csv_path, parse_dates=['time'])
    df = df.sort_values('time').set_index('time')
    df = df[features]

    # Handle zero values in energy
    energy = df['energy'].copy()
    mask = energy == 0
    first_nonzero_idx = energy.ne(0).idxmax()
    mask.loc[:first_nonzero_idx] = False
    energy.loc[mask] = np.nan
    energy = energy.ffill()
    df['energy'] = energy
    df = df.ffill()

    # Normalize
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)

    # Use only the last `window_size` data points for prediction
    latest_window = scaled_data[-window_size:]
    return latest_window, scaler, df[-window_size:].index

In [ ]:
X_input, scaler, last_timestamps = load_latest_data(csv_path, features, window_size)

In [ ]:
X_input = np.expand_dims(X_input, axis=0)

In [ ]:
model = load_model(model_path)

In [ ]:
predictions_scaled = []

In [ ]:
current_input = X_input.copy()

In [ ]:
for _ in range(window_size):
    pred = model.predict(current_input, verbose=0)
    predictions_scaled.append(pred[0])

    # Append prediction and slide window
    current_input = np.append(current_input[:, 1:, :], [[pred]], axis=1)


In [ ]:
predictions_scaled = np.array(predictions_scaled)

In [ ]:
predictions = scaler.inverse_transform(predictions_scaled)

In [ ]:
df = pd.read_csv(csv_path, parse_dates=['time'])
df = df.sort_values('time').set_index('time')
df = df[features]

In [ ]:
energy = df['energy'].copy()
mask = energy == 0
first_nonzero_idx = energy.ne(0).idxmax()
mask.loc[:first_nonzero_idx] = False
energy.loc[mask] = np.nan
energy = energy.ffill()
df['energy'] = energy
df = df.ffill()

In [ ]:
past_values = df[-window_size:]['energy'].values

In [ ]:
start_time = df.index[-1]
time_freq = df.index.inferred_freq or '5min'
predicted_range = pd.date_range(start=start_time + pd.Timedelta(time_freq), periods=window_size, freq=time_freq)


In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(df.index[-window_size:], past_values, label='Input Data (Past 3360)', color='blue')
plt.plot(predicted_range, predictions[:, 0], label='Predicted Data (Next 3360)', color='orange', linestyle='--')
plt.title('Energy Prediction: Past vs Next Week')
plt.xlabel('Time')
plt.ylabel('Energy')
plt.legend()
plt.grid(True)
plt.show()